In [1]:
# %pip install sec-api
# %pip install pdfkit
# %pip install wkhtmltopdf

In [5]:
from sec_api import ExtractorApi
import pdfkit

In [3]:
# https://sec-api.io/login
# https://github.com/janlukasschroeder/sec-api-python/blob/master/README.md#filing-render--download-api
# https://pypi.org/project/sec-api/#filing-render--download-api
extractorApi = ExtractorApi("34686d55d1115b6b415141f1501c0d8c66ce5ef2cc31d77b797dbc028f14e22b")


# 10-K example

# Tesla 10-K filing
filing_url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"

# get the standardized and cleaned text of section 1 "Business"
section1_text = extractorApi.get_section(filing_url_10k, "1", "text")
# get the standardized and cleaned text of section 1A "Risk Factors"
section1a_text = extractorApi.get_section(filing_url_10k, "1A", "text")



# get the original HTML of section 1 "Business"
section1_html = extractorApi.get_section(filing_url_10k, "1", "html")
# get the original HTML of section 1A "Risk Factors"
section1a_html = extractorApi.get_section(filing_url_10k, "1A", "html")

In [6]:
# Configure pdfkit with path to wkhtmltopdf if it's not in PATH
# pdfkit_config = pdfkit.configuration(wkhtmltopdf="/path/to/wkhtmltopdf")

# Combine the HTML content into one large HTML string
combined_html = section1_html + section1a_html

# Output filename for the PDF
output_pdf = 'section1_1a.pdf'

# Convert HTML to PDF
# pdfkit.from_string(combined_html, output_pdf)
pdfkit.from_string(combined_html, output_pdf)

# print(f"The PDF file has been created: {output_pdf}")

True